<span style="color: black; font-weight: bold; font-size: 50px;"> Récuperation de la liste des factures sellsy (anciennes et futures)</span>

In [1]:
from babel.numbers import format_currency
import json
import requests
from requests_oauthlib import OAuth1
from oauthlib.oauth1 import SIGNATURE_PLAINTEXT, SIGNATURE_TYPE_BODY
import pandas as pd


def sellsyFacturation():
    consumer_token = ''
    consumer_secret = ''
    user_token = ''
    user_secret = ''
    url = 'https://apifeed.sellsy.com/0/'
    
    auth = OAuth1(
        consumer_token,
        consumer_secret,
        user_token,
        user_secret,
        signature_method=SIGNATURE_PLAINTEXT,
        signature_type=SIGNATURE_TYPE_BODY,
    )

    payload = {'method': 'DocRows.getList', 'params': {'doctype': 'invoice', 'pagination': {'nbperpage': 2000}}}
    
    response = requests.post(url, data={
        'request': 1,
        'io_mode': 'json',
        'do_in': json.dumps(payload)
    }, auth=auth, headers={'content-type': 'application/json'})
    
    data = response.json()
    
    client = data['response']['result']
    
    liste_ident = []
    liste_doc_thirdname = []
    liste_name = []
    liste_date = []
    liste_doc_totalAmountTaxesFree = []
    liste_email = []
    liste_statut = []
    liste_tel = []
    
    for i in client:
        numero = client[i].get('id', None)
        liste_ident.append(numero)

        doc_thirdname = client[i].get('doc_thirdname', None)
        liste_doc_thirdname.append(doc_thirdname)

        tel = client[i]["thirdDetails"].get('tel', None)
        liste_tel.append(tel)
        
        doc_name = client[i].get('name', None)
        liste_name.append(doc_name)

        doc_date = client[i].get('formatted_doc_created', None)
        liste_date.append(doc_date)

        doc_totalAmountTaxesFree = client[i].get('doc_totalAmountTaxesFree', None)
        if doc_totalAmountTaxesFree is not None:
            montant_formate = format_currency(float(doc_totalAmountTaxesFree), 'EUR', locale='fr_FR')
        else:
            montant_formate = None
        liste_doc_totalAmountTaxesFree.append(montant_formate)

        email = client[i]['thirdDetails'].get('email', None)
        liste_email.append(email)

        statut = client[i]['stepDoc'].get('label', None)
        liste_statut.append(statut)
    df_sellsy = pd.DataFrame(list(zip(
        liste_ident, liste_doc_thirdname,liste_date, liste_tel, liste_email,
        liste_doc_totalAmountTaxesFree, liste_statut, liste_name,
    )), 
    columns=[
        "ID", "Nom complet", "Date", "Mobile", "Email",
        "Montant", "Statut", "Name",
    ])

    return df_sellsy


In [ ]:
currencys=sellsyFacturation()
currencys.to_csv("actuelle.csv", index=False)
currencys

In [ ]:

currencyt = currencys.groupby(["Nom complet","Mobile","Email"],as_index=False)[["Date","Statut","Montant","Name"]].agg('/'.join)
print("c'est bon")
currencyt.to_csv("az.csv", index=False)
currencyt

In [ ]:
import pandas as pd

currency = pd.read_csv("az.csv", encoding='utf-8')

print(currency.head())
currency

In [ ]:
def calculer_somme_montants(montant):
    try:
        montant = montant.replace('€', '').replace('\u202f', '').replace(',', '.').strip()

        montants = [float(x.strip()) for x in montant.split('/')]
        return sum(montants)
    except:
        return 0.0

currency["Montant Sommé"] = currency["Montant"].apply(calculer_somme_montants)

print(currency[["Montant", "Montant Sommé"]].head())

hg = currency.groupby(
    ["Date","Mobile", 'Email', "Name", "Statut"],
    as_index=False
)[["Montant Sommé"]].sum()

try:
    hg.to_csv("az.csv", index=False)
    print("fichier az.csv enregistré")
except Exception as e:
    print(f"Erreur : {e}")

print(currency.head())
print(hg.head())
currency.to_csv("azfsdt.csv", index=False)
currency

<span style="color: black; font-weight: bold; font-size: 50px;"> Filtrage de la facture la plus recente</span>

In [ ]:
import pandas as pd
from datetime import datetime

df = pd.read_csv('azfsdt.csv')

def get_statut_recent(row):
    raw_dates = str(row['Date']).split('/')
    raw_statuts = str(row['Statut']).split('/')

    parsed_dates = []
    for d in raw_dates:
        try:
            parsed_dates.append(datetime.strptime(d.strip(), "%d/%m/%Y"))
        except:
            parsed_dates.append(None)

    clean = [(d, s) for d, s in zip(parsed_dates, raw_statuts) if d is not None]

    if not clean:
        return raw_statuts[-1] if raw_statuts else None
    return max(clean, key=lambda x: x[0])[1]

df['Statut récent'] = df.apply(get_statut_recent, axis=1)

df[['Nom complet', 'Date', 'Statut', 'Statut récent']]
df.to_csv("azfsdtord.csv", index=False)
df

<span style="color: black; font-weight: bold; font-size: 50px;"> Liste des contacts - Fresh</span>

In [ ]:
import requests
import pandas as pd
import time

API_KEY_FRESHWORK = ''
DOMAIN_FRESHWORK = ''
BASE_URL = f'https://{DOMAIN_FRESHWORK}.myfreshworks.com/crm/sales/'

def get_existing_contacts(api_key, domain):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Token token={api_key}'
    }
    all_contacts = {}
    page = 1
    last_contact_id = None

    while True:
        params = {'per_page': 100, 'page': page}
        if last_contact_id:
            params['last_contact_id'] = last_contact_id
        url = f"{BASE_URL}api/contacts/view/31004512042?per_page=100&sort=created_at"

        response = requests.get(url, headers=headers, params=params)
        if not response.ok:
            print(f"Erreur lors de la récupération des données : {response.status_code}")
            return

        response_data = response.json()
        if 'contacts' not in response_data:
            print("Erreur dans la structure de la réponse JSON")
            break

        contacts_data = response_data['contacts']
        if not contacts_data:
            break

        for contact in contacts_data:
            custom_field = contact.get("custom_field", {})
            bitrix_id = custom_field.get('cf_id_bitrix')
            if bitrix_id:
                all_contacts[bitrix_id] = contact

        last_contact_id = contacts_data[-1]['id']
        page += 1
        time.sleep(1)

    print(f"Contacts récupérés avec succès : {len(all_contacts)}")
    return all_contacts

def main():
    existing_contacts = get_existing_contacts(API_KEY_FRESHWORK, DOMAIN_FRESHWORK)
    data = {
        "internal_id": [],
        "first_name": [],
        "last_name": [],
        "email": [],
        "work_number": [],
        "mobile_number": [],
        "phone_number": [],  
        "cf_id_bitrix": [], 
        "updated_at": []
    }

    for contact in existing_contacts.values():
        data["internal_id"].append(contact.get("id", ""))
        data["first_name"].append(contact.get("first_name", ""))
        data["last_name"].append(contact.get("last_name", ""))
        data["email"].append(contact.get("email", ""))
        data["work_number"].append(contact.get("work_number", ""))
        data["mobile_number"].append(contact.get("mobile_number", ""))

        custom_field = contact.get("custom_field", {})
        data["phone_number"].append(custom_field.get("cf_nom_du_prospect", ""))
        data["cf_id_bitrix"].append(custom_field.get("cf_id_bitrix", ""))

        updated_at = contact.get("updated_at", "")
        if updated_at:
            updated_at = updated_at.split("T")[0]
        data["updated_at"].append(updated_at)

    df_combined = pd.DataFrame(data)
    df_combined["updated_at"] = pd.to_datetime(df_combined["updated_at"])
    df_combined["month_year"] = df_combined["updated_at"].dt.to_period('M')
    df_combined = df_combined.sort_values(by="month_year", ascending=False)
    df_combined = df_combined.drop(columns=["month_year"])
    df_combined = df_combined.reset_index(drop=True)
    
    df_combined.to_csv('contacts_rs.csv', index=False, encoding='utf-8')
    
    return df_combined

if __name__ == '__main__':
    contacts_df = main()
    print(contacts_df)


<span style="color: black; font-weight: bold; font-size: 50px;"> Récuperation de la liste des affaires-Freshworks</span>

In [ ]:
import requests
import pandas as pd
import time

API_KEY = ""
DOMAIN = ""
HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"Token token={API_KEY}"
}

def fetch_data(endpoint):
    url = f"https://{DOMAIN}.myfreshworks.com/crm/sales/api/selector/{endpoint}"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return {item["id"]: item["name"] for item in response.json().get(endpoint, [])}
    return {}

def get_pipelines():
    url = f"https://{DOMAIN}.myfreshworks.com/crm/sales/api/selector/deal_pipelines"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return {p["id"]: p["name"] for p in response.json().get("deal_pipelines", [])}
    return {}

def get_pipeline_stages(pipeline_id):
    url = f"https://{DOMAIN}.myfreshworks.com/crm/sales/api/selector/deal_pipelines/{pipeline_id}/deal_stages"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return {s["id"]: s["name"] for s in response.json().get("deal_stages", [])}
    return {}

def get_view_id_by_name(view_name):
    url = f"https://{DOMAIN}.myfreshworks.com/crm/sales/api/deals/filters"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        filters = response.json().get("filters", [])
        for f in filters:
            if f["name"].lower() == view_name.lower():
                return f["id"]
    return None

def get_deals(view_id):
    all_deals = []
    page = 1
    while True:
        url = f"https://{DOMAIN}.myfreshworks.com/crm/sales/api/deals/view/{view_id}"
        params = {"per_page": 100, "page": page}
        response = requests.get(url, headers=HEADERS, params=params)
        if response.status_code != 200:
            return []
        deals = response.json().get("deals", [])
        if not deals:
            break
        all_deals.extend(deals)
        print(f"{len(all_deals)} deals récupérés...")
        page += 1
        time.sleep(1)
    return all_deals

def save_deals_to_csv(deals, deal_stages, lifecycle_stages, owners, pipelines):
    data = {
        "deal_id": [], "name": [], "amount": [], "pipeline": [], "stage": [],
        "lifecycle_stage": [], "owner": [], "created_at": [], "updated_at": []
    }
    for deal in deals:
        pipeline_name = pipelines.get(deal.get("deal_pipeline_id", ""), "Inconnu")
        stage_name = deal_stages.get(deal.get("deal_stage_id", ""), "Inconnu")
        lifecycle_name = lifecycle_stages.get(deal.get("lifecycle_stage_id", ""), "Inconnu")
        owner_name = owners.get(deal.get("owner_id", ""), "Inconnu")

        data["deal_id"].append(deal.get("id", ""))
        data["name"].append(deal.get("name", ""))
        data["amount"].append(deal.get("amount", ""))
        data["pipeline"].append(pipeline_name)
        data["stage"].append(stage_name)
        data["lifecycle_stage"].append(lifecycle_name)
        data["owner"].append(owner_name)
        data["created_at"].append(deal.get("created_at", "").split("T")[0] if deal.get("created_at") else "")
        data["updated_at"].append(deal.get("updated_at", "").split("T")[0] if deal.get("updated_at") else "")
    
    df = pd.DataFrame(data)
    df.to_csv("deals.csv", index=False, encoding="utf-8")
    print(f"{len(df)} deals enregistrés dans 'deals.csv'")

def main():
    print("Pause de 10 min avant la récupération des données pour eviter erreur 429...")
    time.sleep(600)

    view_name = "Open Deals" 
    view_id = get_view_id_by_name(view_name)
    
    if view_id:
        pipelines = get_pipelines()
        deal_stages = {}
        for pipeline_id in pipelines.keys():
            deal_stages.update(get_pipeline_stages(pipeline_id))
        lifecycle_stages = fetch_data("lifecycle_stages")
        owners = fetch_data("owners")
        deals = get_deals(view_id)
        save_deals_to_csv(deals, deal_stages, lifecycle_stages, owners, pipelines)
    else:
        print(f"Filtre '{view_name}' non trouvé.")

if __name__ == "__main__":
    main()


In [ ]:
sellsy = 'deals.csv'
sellsy = pd.read_csv(sellsy)
sellsy

<span style="color: white; font-weight: bold; font-size: 50px;"> Detection des statut SI</span>

In [ ]:
import pandas as pd

df = pd.read_csv('azfsdtord.csv')

filtré = df[df['Name'].str.contains('seance initial|seg|1ère séance|SI seance', case=False, na=False)]

filtré.to_csv("azfsdtordzer.csv", index=False)
filtré

In [ ]:
import pandas as pd

df = pd.read_csv('azfsdtordzer.csv')

filtré = df[df['Statut récent'].str.contains('payée', case=False, na=False)]

filtré.to_csv("azfsdtordzersd.csv", index=False)
filtré

<span style="color: black; font-weight: bold; font-size: 50px;"> Changement De Statut en "Premier bilan effectué" si SEG ou SI est payé sur sellsy</span>

In [12]:
import csv
import requests
import time
from datetime import datetime

API_KEY_FRESHWORK = ''
DOMAIN_FRESHWORK = ''
BASE_URL = f'https://{DOMAIN_FRESHWORK}.myfreshworks.com/crm/sales/'

HEADERS = {
    'Content-Type': 'application/json',
    'Authorization': f'Token token={API_KEY_FRESHWORK}'
}

DEFAULT_DEAL_STAGE_ID = 31000543861
LOG_FILE = "log_traitement.txt"

def log(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(LOG_FILE, "a", encoding="utf-8") as log_file:
        log_file.write(f"[{timestamp}] {message}\n")
    print(message)

def get_existing_contacts():
    log("Début récupération des contacts existants...")
    all_contacts = {}
    page = 1
    while True:
        url = f"{BASE_URL}api/contacts/view/31004512042?sort=created_at&per_page=100&page={page}"
        response = requests.get(url, headers=HEADERS)
        if response.status_code != 200:
            log(f"Erreur contacts: {response.text}")
            return {}
        contacts_data = response.json().get("contacts", [])
        if not contacts_data:
            break
        for contact in contacts_data:
            email = contact.get("email")
            if email:
                all_contacts[email.strip().lower()] = contact["id"]
        page += 1
        time.sleep(1)
    log(f"{len(all_contacts)} contacts récupérés.")
    return all_contacts

def get_existing_deal(contact_id):
    url = f"{BASE_URL}api/contacts/{contact_id}/deals"
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        log(f"Erreur récupération des affaires pour contact {contact_id}")
        return None
    deals = response.json().get("deals", [])
    for deal in deals:
        try:
            amount = float(deal.get("amount", 0) or 0)
        except:
            amount = 0
        if amount > 0:
            return deal
    return None

def update_deal_stage(deal_id, new_stage_id=DEFAULT_DEAL_STAGE_ID):
    url = f"{BASE_URL}api/deals/{deal_id}"
    payload = {
        "deal": {
            "deal_stage_id": new_stage_id
        }
    }
    response = requests.put(url, headers=HEADERS, json=payload)
    if response.status_code == 200:
        log(f"Affaire {deal_id} mise à jour vers 'Premier bilan effectué'")
    else:
        log(f"Erreur mise à jour affaire {deal_id}: {response.text}")

def main():
    log("Pause 1/2 : 1 heure avant récupération des contacts...")
    time.sleep(3600)

    existing_contacts = get_existing_contacts()

    with open('azfsdtordzersd.csv', 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        bitrix_contacts = list(reader)

    log("Pause 2/2 : 1 heure avant mise à jour des affaires...")
    time.sleep(3600)

    for contact in bitrix_contacts:
        email = contact.get('Email', '').strip().lower()
        if not email:
            continue
        if email in existing_contacts:
            contact_id = existing_contacts[email]
            deal = get_existing_deal(contact_id)
            if deal:
                update_deal_stage(deal['id'])
        time.sleep(1)

    log("Traitement terminé.")

if __name__ == "__main__":
    main()


Pause 1/2 : 1 heure avant récupération des contacts...
Début récupération des contacts existants...
38730 contacts récupérés.
Pause 2/2 : 1 heure avant mise à jour des affaires...
Affaire 31003593093 mise à jour vers 'Premier bilan effectué'
Affaire 31003144848 mise à jour vers 'Premier bilan effectué'
Affaire 31003601984 mise à jour vers 'Premier bilan effectué'
Affaire 31003593084 mise à jour vers 'Premier bilan effectué'
Affaire 31005928505 mise à jour vers 'Premier bilan effectué'
Affaire 31003031391 mise à jour vers 'Premier bilan effectué'
Affaire 31003516195 mise à jour vers 'Premier bilan effectué'
Affaire 31003581241 mise à jour vers 'Premier bilan effectué'
Affaire 31003031198 mise à jour vers 'Premier bilan effectué'
Affaire 31003618186 mise à jour vers 'Premier bilan effectué'
Affaire 31003030358 mise à jour vers 'Premier bilan effectué'
Affaire 31003031332 mise à jour vers 'Premier bilan effectué'
Affaire 31003031492 mise à jour vers 'Premier bilan effectué'
Affaire 310030

<span style="color: black; font-weight: bold; font-size: 50px;"> Detection des statut Retard / Impayé</span>

In [ ]:
import pandas as pd

df = pd.read_csv('azfsdtord.csv')

filtré = df[df['Statut récent'].str.contains('Retard|Impayé', case=False, na=False)]

filtré.to_csv("azfsdtordzer_impayer.csv", index=False)
filtré

In [14]:
import csv
import requests
import time

API_KEY_FRESHWORK = ''
DOMAIN_FRESHWORK = ''
BASE_URL = f'https://{DOMAIN_FRESHWORK}.myfreshworks.com/crm/sales/'

HEADERS = {
    'Content-Type': 'application/json',
    'Authorization': f'Token token={API_KEY_FRESHWORK}'
}

DEFAULT_DEAL_STAGE_ID = 31000543867

def get_existing_contacts():
    print("\nRécupération des contacts existants (email)...")
    all_contacts = {}
    page = 1
    while True:
        url = f"{BASE_URL}api/contacts/view/31004512042?sort=created_at&per_page=100&page={page}"
        response = requests.get(url, headers=HEADERS)
        if response.status_code != 200:
            print(f"Erreur contacts: {response.text}")
            return {}
        contacts_data = response.json().get("contacts", [])
        if not contacts_data:
            break
        for contact in contacts_data:
            email = contact.get("email")
            if email:
                all_contacts[email.strip().lower()] = contact["id"]
        page += 1
        time.sleep(1)
    print(f"{len(all_contacts)} contacts récupérés.")
    return all_contacts

def get_existing_deal(contact_id):
    url = f"{BASE_URL}api/contacts/{contact_id}/deals"
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        print(f"Erreur récupération des affaires pour contact {contact_id}")
        return None
    deals = response.json().get("deals", [])
    for deal in deals:
        try:
            amount = float(deal.get("amount", 0) or 0)
        except:
            amount = 0
        if amount > 0:
            return deal
    return None

def update_deal_stage(deal_id, new_stage_id=DEFAULT_DEAL_STAGE_ID):
    url = f"{BASE_URL}api/deals/{deal_id}"
    payload = {
        "deal": {
            "deal_stage_id": new_stage_id
        }
    }
    response = requests.put(url, headers=HEADERS, json=payload)
    if response.status_code == 200:
        print(f"Affaire {deal_id} mise à jour vers 'Retard / Impayé'")
    else:
        print(f"Erreur mise à jour affaire {deal_id}: {response.text}")

def main():
    print("Pause 1/2 : 1 heure avant la récupération des contacts...")
    time.sleep(60)

    existing_contacts = get_existing_contacts()

    with open('azfsdtordzer_impayer.csv', 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        bitrix_contacts = list(reader)

    print("Pause 2/2 : 1 heure avant la mise à jour des affaires...")
    time.sleep(3600)

    for contact in bitrix_contacts:
        email = contact.get('Email', '').strip().lower()
        if not email:
            continue
        if email in existing_contacts:
            contact_id = existing_contacts[email]
            deal = get_existing_deal(contact_id)
            if deal:
                update_deal_stage(deal['id'])
        time.sleep(1)

if __name__ == "__main__":
    main()


Pause 1/2 : 1 heure avant la récupération des contacts...

Récupération des contacts existants (email)...
38730 contacts récupérés.
Pause 2/2 : 1 heure avant la mise à jour des affaires...
Affaire 31004903613 mise à jour vers 'Retard / Impayé'
Affaire 31003046178 mise à jour vers 'Retard / Impayé'
Affaire 31003046092 mise à jour vers 'Retard / Impayé'
Affaire 31003046195 mise à jour vers 'Retard / Impayé'
Affaire 31003828996 mise à jour vers 'Retard / Impayé'
Affaire 31003046157 mise à jour vers 'Retard / Impayé'
Affaire 31005213442 mise à jour vers 'Retard / Impayé'
Affaire 31003507382 mise à jour vers 'Retard / Impayé'
Affaire 31005891536 mise à jour vers 'Retard / Impayé'
Affaire 31005904425 mise à jour vers 'Retard / Impayé'
Affaire 31004029843 mise à jour vers 'Retard / Impayé'
Affaire 31005949572 mise à jour vers 'Retard / Impayé'
Affaire 31003030504 mise à jour vers 'Retard / Impayé'
Affaire 31005756471 mise à jour vers 'Retard / Impayé'
Affaire 31005949540 mise à jour vers 'Ret

<span style="color: black; font-weight: bold; font-size: 50px;"> Suivi de l’évolution des statuts des factures MPA Facture Annulée</span>

In [ ]:
import pandas as pd

df = pd.read_csv('azfsdtord.csv')

filtré = df[df['Statut récent'].str.contains('Annulée', case=False, na=False)]

filtré.to_csv("azfsdtordzer_Annulée.csv", index=False)
filtré

In [16]:
import csv
import requests
import time

API_KEY_FRESHWORK = ''
DOMAIN_FRESHWORK = ''
BASE_URL = f'https://{DOMAIN_FRESHWORK}.myfreshworks.com/crm/sales/'

HEADERS = {
    'Content-Type': 'application/json',
    'Authorization': f'Token token={API_KEY_FRESHWORK}'
}

DEFAULT_DEAL_STAGE_ID = 31001026962

def get_existing_contacts():
    print("\nRécupération des contacts existants (email)...")
    all_contacts = {}
    page = 1
    while True:
        url = f"{BASE_URL}api/contacts/view/31004512042?sort=created_at&per_page=100&page={page}"
        response = requests.get(url, headers=HEADERS)
        if response.status_code != 200:
            print(f"Erreur contacts: {response.text}")
            return {}
        contacts_data = response.json().get("contacts", [])
        if not contacts_data:
            break
        for contact in contacts_data:
            email = contact.get("email")
            if email:
                all_contacts[email.strip().lower()] = contact["id"]
        page += 1
        time.sleep(1)
    print(f"{len(all_contacts)} contacts récupérés.")
    return all_contacts

def get_existing_deal(contact_id):
    url = f"{BASE_URL}api/contacts/{contact_id}/deals"
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        print(f"Erreur récupération des affaires pour contact {contact_id}")
        return None
    deals = response.json().get("deals", [])
    for deal in deals:
        try:
            amount = float(deal.get("amount", 0) or 0)
        except:
            amount = 0
        if amount > 0:
            return deal
    return None

def update_deal_stage(deal_id, new_stage_id=DEFAULT_DEAL_STAGE_ID):
    url = f"{BASE_URL}api/deals/{deal_id}"
    payload = {
        "deal": {
            "deal_stage_id": new_stage_id
        }
    }
    response = requests.put(url, headers=HEADERS, json=payload)
    if response.status_code == 200:
        print(f"Affaire {deal_id} mise à jour vers 'MPA Annulée'")
    else:
        print(f"Erreur mise à jour affaire {deal_id}: {response.text}")

def main():
    print("Pause 1/2 : 1 heure avant la récupération des contacts...")
    time.sleep(3600)

    existing_contacts = get_existing_contacts()

    with open('azfsdtordzer_Annulée.csv', 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        bitrix_contacts = list(reader)

    print("Pause 2/2 : 1 heure avant la mise à jour des affaires...")
    time.sleep(1)

    for contact in bitrix_contacts:
        email = contact.get('Email', '').strip().lower()
        if not email:
            continue
        if email in existing_contacts:
            contact_id = existing_contacts[email]
            deal = get_existing_deal(contact_id)
            if deal:
                update_deal_stage(deal['id'])
        time.sleep(1)

if __name__ == "__main__":
    main()


Pause 1/2 : 1 heure avant la récupération des contacts...

Récupération des contacts existants (email)...
38731 contacts récupérés.
Pause 2/2 : 1 heure avant la mise à jour des affaires...
Affaire 31004052729 mise à jour vers 'MPA Annulée'
Affaire 31004069719 mise à jour vers 'MPA Annulée'
Affaire 31004070830 mise à jour vers 'MPA Annulée'
Affaire 31005206282 mise à jour vers 'MPA Annulée'
Affaire 31003665581 mise à jour vers 'MPA Annulée'
Affaire 31005216667 mise à jour vers 'MPA Annulée'
Affaire 31005214882 mise à jour vers 'MPA Annulée'
Affaire 31003030495 mise à jour vers 'MPA Annulée'
Affaire 31003869605 mise à jour vers 'MPA Annulée'
Affaire 31005214924 mise à jour vers 'MPA Annulée'
Affaire 31005205920 mise à jour vers 'MPA Annulée'
Affaire 31003031449 mise à jour vers 'MPA Annulée'
Affaire 31005209956 mise à jour vers 'MPA Annulée'
Affaire 31004031915 mise à jour vers 'MPA Annulée'
Affaire 31003514541 mise à jour vers 'MPA Annulée'
Affaire 31005370691 mise à jour vers 'MPA Annu

<span style="color: white; font-weight: bold; font-size: 50px;">Acuity</span>

In [17]:
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
import datetime
import time

USER_ID = ''
API_KEY = ''

def get_appointments_in_range(start_date, end_date):
    url = f"https://acuityscheduling.com/api/v1/appointments?minDate={start_date}&maxDate={end_date}&showall=true"
    response = requests.get(url, auth=HTTPBasicAuth(USER_ID, API_KEY))
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erreur API pour la période {start_date} à {end_date} : {response.status_code}")
        return []

def get_appointment_by_id(appointment_id):
    url = f"https://acuityscheduling.com/api/v1/appointments/{appointment_id}"
    response = requests.get(url, auth=HTTPBasicAuth(USER_ID, API_KEY))
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erreur lors de la récupération du RDV ID {appointment_id}")
        return None

def daterange(start_date, end_date, step_days=7):
    while start_date <= end_date:
        yield start_date, min(end_date, start_date + datetime.timedelta(days=step_days - 1))
        start_date += datetime.timedelta(days=step_days)

def collect_all_appointments():
    all_data = []
    seen_ids = set()
    start = datetime.date(2025, 4, 1)
    end = datetime.date(2025, 12, 31)

    print("Début de la collecte des rendez-vous")

    for start_date, end_date in daterange(start, end):
        print(f"\nPériode : {start_date} à {end_date}")
        appointments = get_appointments_in_range(start_date, end_date)
        print(f"{len(appointments)} rendez-vous trouvés.")

        for appt in appointments:
            appt_id = appt.get("id")
            if appt_id in seen_ids:
                continue
            seen_ids.add(appt_id)

            details = get_appointment_by_id(appt_id)
            if details:
                appt["scheduledBy"] = details.get("scheduledBy", "Non spécifié")
            else:
                appt["scheduledBy"] = "Non spécifié"

            nom = f"{appt.get('lastName', '')} {appt.get('firstName', '')}".strip()
            date = appt.get('date', 'Date inconnue')
            heure = appt.get('time', 'Heure inconnue')
            type_rdv = appt.get('type', 'Type inconnu')
            calendrier = appt.get('calendar', 'Calendrier inconnu')

            print(f"  RDV ID {appt_id} | {nom} - {date} {heure} - {type_rdv} - Calendrier: {calendrier} - Créé par: {appt['scheduledBy']}")

            all_data.append(appt)
            time.sleep(0.2)

        time.sleep(0.5)

    print(f"\nCollecte terminée. Total RDV : {len(all_data)}")
    return all_data

def to_dataframe(data):
    rows = []
    for item in data:
        rows.append({
            "Nom": f"{item.get('lastName', '')} {item.get('firstName', '')}".strip(),
            "Email": item.get('email', ''),
            "Téléphone": item.get('phone', ''),
            "Date de création": item.get('datetimeCreated', ''),
            "Date du RDV": item.get('date', ''),
            "Heure": item.get('time', ''),
            "Type de RDV": item.get('type', ''),
            "Calendrier": item.get('calendar', ''),
            "Créé par": item.get('scheduledBy', ''),
        })
    return pd.DataFrame(rows)

def save_csv(df):
    filename = "acuity_rdv_2024_2025.csv"
    df.to_csv(filename, index=False, encoding="utf-8")
    print(f"\nFichier exporté : {filename}")

if __name__ == "__main__":
    data = collect_all_appointments()
    df = to_dataframe(data)
    save_csv(df)


Début de la collecte des rendez-vous

Période : 2025-04-01 à 2025-04-07
47 rendez-vous trouvés.
  RDV ID 1444404503 | Renoux Caroline - April 7, 2025 16:30 - Mise en place administrative accompagnement Oser le changement - Calendrier: Aurélie en télé-présentiel - Créé par: evaluation@institutadios.com
  RDV ID 1442061675 | Mendel Antoine - April 7, 2025 16:00 - Session de 3h en visio-consultation à l'unité SarahNacass.com - Calendrier: Praticien(ne) Sarah Nacass via Zoom - Créé par: contact@sarahnacass.com
  RDV ID 1443209596 | Briand Vincent - April 7, 2025 14:00 - Activation du changement de 3h avec un(e) praticien(ne) expérimenté(e) sur place n°rdv.oserlechangement - Calendrier: Praticien(ne) Oser le Changement en activation du changement sur Place - Créé par: accueil@institutadios.com
  RDV ID 1445720414 | Lacarotte Jo - April 7, 2025 13:00 - Session de 3h sur place à l'unité - Calendrier: Praticien(ne) Sarah Nacass sur Place - Créé par: contact@sarahnacass.com
  RDV ID 1441572965 

In [ ]:
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
import datetime
import time

df1 = pd.read_csv("acuity_rdv_2024_2025.csv")
df1["Mobile"]=df1["Téléphone"]
del df1["Téléphone"]
print(f"\033[31mNombre de rendez-vous total du 1/1/2024-2025 : {len(df1)}\033[0m")
df1.to_csv("acuity_appointments_MPA_2025.csv", index=False)
df1

In [ ]:
import pandas as pd


df1 = pd.read_csv("acuity_rdv_2024_2025.csv")


df1 = df1[~df1["Nom"].str.contains("carotte|test", case=False, na=False)]

b = df1.loc[df1["Type de RDV"].str.contains(
    "Mise en place ad",
    regex=False
)]

print(f"\033[31mNombre de rendez-vous trouvés : {len(b)}\033[0m")

b.to_csv("MPA_prévue.csv", index=False)

b


<span style="color: black; font-weight: bold; font-size: 50px;"> Suivi de l’évolution des statuts des factures MPA</span>

In [ ]:
import pandas as pd

acuity = pd.read_csv('MPA_prévue.csv', dtype=str)
sellsy = pd.read_csv('azfsdtord.csv', dtype=str)

acuity['Email'] = acuity['Email'].str.strip().str.lower()
sellsy['Email'] = sellsy['Email'].str.strip().str.lower()

acuity_unique = acuity[~acuity['Email'].isin(sellsy['Email'])]

print(f"{len(acuity_unique)} emails trouvés dans Acuity mais absents de Sellsy :")
print(acuity_unique[['Email']].drop_duplicates())

acuity_unique.to_csv('emails_absents_de_sellsy_MPA.csv', index=False)
acuity_unique

In [25]:
import csv
import requests
import time
import uuid

API_KEY_FRESHWORK = ''
DOMAIN_FRESHWORK = ''
BASE_URL = f'https://{DOMAIN_FRESHWORK}.myfreshworks.com/crm/sales/'

HEADERS = {
    'Content-Type': 'application/json',
    'Authorization': f'Token token={API_KEY_FRESHWORK}'
}

DEFAULT_DEAL_STAGE_ID = 31000543862

def generate_cf_id_bitrix():
    """Génère un identifiant unique pour cf_id_bitrix."""
    return str(uuid.uuid4())

def truncate(text, max_length):
    return text[:max_length] if text else ""

def get_existing_contacts():
    print("\nRécupération des contacts existants (email)...")
    all_contacts = {}
    page = 1
    while True:
        url = f"{BASE_URL}api/contacts/view/31004512042?sort=created_at&per_page=100&page={page}"
        response = requests.get(url, headers=HEADERS)
        if response.status_code != 200:
            print(f"Erreur contacts: {response.text}")
            return {}
        contacts_data = response.json().get("contacts", [])
        if not contacts_data:
            break
        for contact in contacts_data:
            email = contact.get("email")
            if email:
                all_contacts[email.strip().lower()] = contact["id"]
        page += 1
        time.sleep(1)
    print(f"{len(all_contacts)} contacts récupérés.")
    return all_contacts

def get_existing_deal(contact_id):
    url = f"{BASE_URL}api/contacts/{contact_id}/deals"
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        print(f"Erreur récupération des affaires pour contact {contact_id}")
        return None
    deals = response.json().get("deals", [])
    for deal in deals:
        try:
            amount = float(deal.get("amount", 0) or 0)
        except:
            amount = 0
        if amount > 0:
            return deal
    return None

def update_deal_stage(deal_id, new_stage_id=DEFAULT_DEAL_STAGE_ID):
    url = f"{BASE_URL}api/deals/{deal_id}"
    payload = {
        "deal": {
            "deal_stage_id": new_stage_id
        }
    }
    response = requests.put(url, headers=HEADERS, json=payload)
    if response.status_code == 200:
        print(f"Affaire {deal_id} mise à jour vers 'MPA prévue'")
    else:
        print(f"Erreur mise à jour affaire {deal_id}: {response.text}")

def create_deal_for_contact(contact_id):
    url = f"{BASE_URL}api/deals"
    payload = {
        "deal": {
            "name": "Affaire générée automatiquement",
            "contacts_added_list": [contact_id],
            "amount": 0,
            "deal_stage_id": DEFAULT_DEAL_STAGE_ID
        }
    }
    response = requests.post(url, headers=HEADERS, json=payload)
    if response.status_code == 201:
        print(f"Affaire créée pour le contact {contact_id}")
        return response.json().get("deal", {})
    else:
        print(f"Erreur création affaire pour le contact {contact_id}: {response.text}")
        return None

def create_contact(row):
    """Créer un contact depuis le CSV avec tous les champs nécessaires."""
    email = row.get("Email", "").strip().lower()
    last_name = row.get("Nom", "").strip()
    action = row.get("Type de RDV", "").strip()
    mobile = row.get("Téléphone", "").strip()
    date_creation = row.get("Date de création", "").strip()
    cf_id_bitrix = row.get("cf_id_bitrix", "").strip() or generate_cf_id_bitrix()

    payload = {
        "contact": {
            "last_name": last_name,
            "email": email,
            "mobile_number": mobile,
            "custom_field": {
                "cf_id_bitrix": cf_id_bitrix,
                "cf_action_initiale_du_prospect": action,
                "cf_date_de_creation": date_creation,
            }
        }
    }

    print(f"Création du contact : {last_name} <{email}> (bitrix ID: {cf_id_bitrix})")
    url = f"{BASE_URL}api/contacts"
    response = requests.post(url, headers=HEADERS, json=payload)
    if response.status_code == 201:
        contact_id = response.json()['contact']['id']
        print(f"Contact créé (ID: {contact_id})")
        return contact_id
    else:
        print(f"Erreur création contact : {response.text}")
        return None


def main():
    existing_contacts = get_existing_contacts()

    with open('emails_absents_de_sellsy_MPA.csv', 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        bitrix_contacts = list(reader)

    for contact in bitrix_contacts:
        email = contact.get('Email', '').strip().lower()
        if not email:
            continue

        if email in existing_contacts:
            contact_id = existing_contacts[email]
            deal = get_existing_deal(contact_id)
            if deal:
                update_deal_stage(deal['id'])
            else:
                create_deal_for_contact(contact_id)
        else:
            contact_id = create_contact(contact)
            if contact_id:
                existing_contacts[email] = contact_id 
                create_deal_for_contact(contact_id)

        time.sleep(1)


if __name__ == "__main__":
    main()



Récupération des contacts existants (email)...
38729 contacts récupérés.
Erreur création affaire pour le contact 31118050766: {"users":[{"id":31000890026,"display_name":"James Aurelie","email":"accueil@sarahnacass.com","is_active":true,"work_number":null,"mobile_number":null}],"deal":{"id":31006921089,"name":"Affaire générée automatiquement","amount":"0.0","base_currency_amount":"0.0","expected_close":null,"closed_date":null,"stage_updated_time":"2025-05-12T15:55:22+02:00","custom_field":{"cf_transaction":null,"cf_activit":null,"cf_statut_du_paiement":null,"cf_statut_de_livraison":null,"cf_types":null,"cf_sources":null,"cf_information_sur_la_source":null,"cf_chemin_du_client":null,"cf_url_de_la_source":null,"cf_score":null,"cf_objectif_formul":null,"cf_date_de_naissance":null,"cf_budget_journalier_utm_medium":null,"cf_action_initiale_du_prospect":null,"cf_etapes_de_la_transaction":null,"cf_date_de_creation":null,"cf_etapes_de_formation":null,"cf_id_bitrix":null,"cf_mobile":null,"cf_em

<span style="color: black; font-weight: bold; font-size: 50px;"> Suivi de l’évolution des programmes payée montant supérieur a 200€ -> en bonne voie</span>

In [ ]:
import pandas as pd

df = pd.read_csv('azfsdtord.csv')

filtré = df[
    df['Name'].str.contains("programme", case=False, na=False) &
    df['Statut récent'].str.contains("payée", case=False, na=False)
]

filtré.to_csv("programmes.csv", index=False)


print(f"{len(filtré)} lignes filtrées avec 'programme' dans 'Type de RDV'")
filtré

In [1]:
import csv
import requests
import time

API_KEY_FRESHWORK = ''
DOMAIN_FRESHWORK = ''
BASE_URL = f'https://{DOMAIN_FRESHWORK}.myfreshworks.com/crm/sales/'

HEADERS = {
    'Content-Type': 'application/json',
    'Authorization': f'Token token={API_KEY_FRESHWORK}'
}

DEFAULT_DEAL_STAGE_ID = 31000584324

def get_existing_contacts():
    print("\nRécupération des contacts existants (email)...")
    all_contacts = {}
    page = 1
    while True:
        url = f"{BASE_URL}api/contacts/view/31004512042?sort=created_at&per_page=100&page={page}"
        response = requests.get(url, headers=HEADERS)
        if response.status_code != 200:
            print(f"Erreur contacts: {response.text}")
            return {}
        contacts_data = response.json().get("contacts", [])
        if not contacts_data:
            break
        for contact in contacts_data:
            email = contact.get("email")
            if email:
                all_contacts[email.strip().lower()] = contact["id"]
        page += 1
        time.sleep(1)
    print(f"{len(all_contacts)} contacts récupérés.")
    return all_contacts

def get_existing_deal(contact_id):
    url = f"{BASE_URL}api/contacts/{contact_id}/deals"
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        print(f"Erreur récupération des affaires pour contact {contact_id}")
        return None
    deals = response.json().get("deals", [])
    for deal in deals:
        try:
            amount = float(deal.get("amount", 0) or 0)
        except:
            amount = 0
        if amount > 0:
            return deal
    return None

def update_deal_stage(deal_id, new_stage_id=DEFAULT_DEAL_STAGE_ID):
    url = f"{BASE_URL}api/deals/{deal_id}"
    payload = {
        "deal": {
            "deal_stage_id": new_stage_id
        }
    }
    response = requests.put(url, headers=HEADERS, json=payload)
    if response.status_code == 200:
        print(f"Affaire {deal_id} mise à jour vers 'Programme'")
    else:
        print(f"Erreur mise à jour affaire {deal_id}: {response.text}")

def main():
    print("Pause 1/2 : 1 heure avant la récupération des contacts...")
    time.sleep(1)

    existing_contacts = get_existing_contacts()

    with open('programmes.csv', 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        bitrix_contacts = list(reader)

    print("Pause 2/2 : 1 heure avant la mise à jour des affaires...")
    time.sleep(1)

    for contact in bitrix_contacts:
        email = contact.get('Email', '').strip().lower()
        if not email:
            continue
        if email in existing_contacts:
            contact_id = existing_contacts[email]
            deal = get_existing_deal(contact_id)
            if deal:
                update_deal_stage(deal['id'])
        time.sleep(1)

if __name__ == "__main__":
    main()


Pause 1/2 : 1 heure avant la récupération des contacts...

Récupération des contacts existants (email)...
38729 contacts récupérés.
Pause 2/2 : 1 heure avant la mise à jour des affaires...
Affaire 31005403555 mise à jour vers 'Programme'
Affaire 31005206149 mise à jour vers 'Programme'
Affaire 31005214867 mise à jour vers 'Programme'
Affaire 31003841096 mise à jour vers 'Programme'
Affaire 31005843403 mise à jour vers 'Programme'
Affaire 31003030457 mise à jour vers 'Programme'
Affaire 31003031920 mise à jour vers 'Programme'
Affaire 31003031502 mise à jour vers 'Programme'
Affaire 31003031173 mise à jour vers 'Programme'
Affaire 31005928505 mise à jour vers 'Programme'
Affaire 31003031391 mise à jour vers 'Programme'
Affaire 31005756337 mise à jour vers 'Programme'
Affaire 31003598247 mise à jour vers 'Programme'
Affaire 31003031198 mise à jour vers 'Programme'
Affaire 31005756220 mise à jour vers 'Programme'
Affaire 31005211251 mise à jour vers 'Programme'
Affaire 31005209257 mise à 